# Compare CLM5 and CLM6 parameter files 

In [44]:
import numpy as np
import pandas as pd
import xarray as xr

In [86]:
xr.set_options(display_expand_attrs=False, display_expand_data=False);
pd.set_option('display.max_rows', 60);

In [116]:
clm5_paramfile = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles/COUP0000.nc"
clm6_paramfile = "/glade/work/wwieder/modified_inputs/ctsm5.3.041.Nfix_params.v13.c250221_upplim250.nc"

clm5_pfile = xr.open_dataset(clm5_paramfile, decode_timedelta=False)
clm6_pfile = xr.open_dataset(clm6_paramfile, decode_timedelta=False)

clm5_params = set(clm5_pfile.data_vars)
clm6_params = set(clm6_pfile.data_vars)

all_params = clm5_params | clm6_params
in_both = clm5_params & clm6_params
clm5_only = clm5_params - clm6_params
clm6_only = clm6_params - clm5_params

---

In [202]:
diff_dim_params = set()  # the dimensions of the parameter changed between CLM5 and CLM6 (made it PFT-specific)
for p in all_params:
    if (p in clm5_params) and (p in clm6_params):
        if clm5_pfile[p].dims != clm6_pfile[p].dims:
            print(p, "- diff dims,", clm5_pfile[p].dims, "->", clm6_pfile[p].dims)
            diff_dim_params.add(p)

r_mort - diff dims, () -> ('pft',)
ndays_on - diff dims, () -> ('pft',)
wc2wjb0 - diff dims, () -> ('pft',)
jmaxb0 - diff dims, () -> ('pft',)


In [203]:
for p in diff_dim_params:
    print("##",p,clm5_pfile[p].attrs["long_name"])
    print("CLM5:", clm5_pfile[p].values)
    print("CLM6:", clm6_pfile[p].values)
    print()

## r_mort Mortality rate
CLM5: 0.02
CLM6: [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]

## wc2wjb0 The baseline ratio of rubisco limited rate vs light limited photosynthetic rate (Wc:Wj)
CLM5: 0.8054
CLM6: [0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054
 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054
 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054
 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054
 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054
 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054 0.8054
 0.8054 0.8

---

In [204]:
def param_set_to_dataframe(this_set, this_pfile):
    param = list()
    long_name = list()
    units = list()

    for p in sorted(list(this_set)):
        param.append(p)
        long_name.append(this_pfile[p].attrs["long_name"])
        if "units" in this_pfile[p].attrs:
            units.append(this_pfile[p].attrs["units"])
        else:
            units.append("")

    this_dataframe = pd.DataFrame(
        data=np.array([param, long_name, units]).T,
        columns=["param", "long_name", "units"]
    )
    return this_dataframe

In [205]:
clm5_only_pd = param_set_to_dataframe(clm5_only, clm5_pfile)

print(f"number of paramters in CLM5 and not in CLM6: {len(clm5_only)}")
clm5_only_pd;

number of paramters in CLM5 and not in CLM6: 18


In [206]:
clm6_only_pd = param_set_to_dataframe(clm6_only, clm6_pfile)

print(f"number of paramters in CLM6 and not in CLM5: {len(clm6_only)}")
clm6_only_pd;

number of paramters in CLM6 and not in CLM5: 77


In [207]:
in_both_clm5_pd = param_set_to_dataframe(in_both, clm5_pfile)
in_both_clm6_pd = param_set_to_dataframe(in_both, clm6_pfile)

In [208]:
in_both_clm5_pd.loc[in_both_clm5_pd["param"]=="zlnd"]["long_name"].item()

'Momentum roughness length for soils, glacier, wetland'

In [209]:
in_both_clm6_pd.loc[in_both_clm6_pd["param"]=="zlnd"]["long_name"].item()

'Momentum roughness length for soils and wetland'

In [210]:
clm5_only_pd["version"] = "CLM5"
clm6_only_pd["version"] = "CLM6"
in_both_clm6_pd["version"] = "both"
in_both_clm5_pd["version"] = "both"

clm5_only_pd["flag"] = ""
clm6_only_pd["flag"] = ""
in_both_clm6_pd["flag"] = ""
in_both_clm5_pd["flag"] = ""

clm5_only_pd["notes"] = ""
clm6_only_pd["notes"] = ""
in_both_clm6_pd["notes"] = ""
in_both_clm5_pd["notes"] = ""

combined_pd = pd.concat([in_both_clm6_pd, clm6_only_pd, clm5_only_pd], ignore_index=True)
combined_pd = combined_pd.sort_values(by="long_name")

In [211]:
duplicate_long_names = combined_pd[combined_pd.duplicated(subset=["long_name"], keep=False)]

temp = duplicate_long_names[duplicate_long_names["version"]!="both"].sort_values(by=["long_name", "version"])
temp = temp[~temp["param"].str.contains("scvng")]

duplicate_clm5 = temp[temp["version"]=="CLM5"]
duplicate_clm6 = temp[temp["version"]=="CLM6"]

clm5_to_clm6_rename = dict(zip(duplicate_clm5["param"], duplicate_clm6["param"]))
clm5_to_clm6_rename

{'tau_l1': 'bgc_tau_l1',
 'tau_l2_l3': 'bgc_tau_l2_l3',
 'tau_s1': 'bgc_tau_s1',
 'tau_s2': 'bgc_tau_s2',
 'tau_s3': 'bgc_tau_s3',
 'cn_s1_bgc': 'bgc_cn_s1',
 'cn_s2_bgc': 'bgc_cn_s2',
 'cn_s3_bgc': 'bgc_cn_s3',
 'cwd_fcel': 'bgc_cwd_fcel',
 'rf_s2s3_bgc': 'bgc_rf_s2s3',
 'rf_l1s1_bgc': 'bgc_rf_l1s1',
 'rf_s2s1_bgc': 'bgc_rf_s2s1',
 'rf_s3s1_bgc': 'bgc_rf_s3s1',
 'rf_cwdl2_bgc': 'rf_cwdl2',
 'rf_cwdl3_bgc': 'bgc_rf_cwdl3',
 'rf_l3s2_bgc': 'bgc_rf_l3s2',
 'rf_l2s1_bgc': 'bgc_rf_l2s1'}

---

In [232]:
new_pfile = clm5_pfile.copy(deep=True)

for p in clm5_params:
    if p in clm5_to_clm6_rename.keys():
        new_pfile[p] = clm6_pfile[clm5_to_clm6_rename[p]]
    
    elif (p in clm6_params) and (clm5_pfile[p].dims == clm6_pfile[p].dims):
        new_pfile[p] = clm6_pfile[p]

new_pfile.attrs["note"] = "BGB: "+clm6_paramfile

new_pfile.to_netcdf("/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles/mod_ctsm5.3.041.Nfix_params.v13.c250221_upplim250.nc")

In [226]:
diff_params = []
for p in clm5_pfile.data_vars:
    if p in new_pfile.data_vars:
        if not np.allclose(clm5_pfile[p].values, new_pfile[p].values, equal_nan=True):
            diff_params.append(p)
    else:
        diff_params.append(p)

print(f"Number of variables with differences: {len(diff_params)}")
print(diff_params)

Number of variables with differences: 31
['akc_active', 'akn_active', 'c3psn', 'cc_leaf', 'cc_other', 'ekc_active', 'ekn_active', 'fm_droot', 'fm_dstem', 'fm_leaf', 'fm_lroot', 'fm_lstem', 'fm_other', 'fm_root', 'froot_leaf', 'fsr_pft', 'kc_nonmyc', 'kn_nonmyc', 'leafcn', 'medlynslope', 'prh30', 'rf_cwdl2_bgc', 'rf_cwdl3_bgc', 'rswf_max', 'rswf_min', 'scvng_fct_mlt_sf', 'slatop', 'snw_rds_refrz', 'xdrdt', 'zlnd', 'zsno']


In [230]:
clm6_paramfile

'/glade/work/wwieder/modified_inputs/ctsm5.3.041.Nfix_params.v13.c250221_upplim250.nc'

---

In [251]:
clm5_oaat_csv = pd.read_csv("/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/02_set-up-ensemble/clm5_paramranges.csv")
clm5_nml_params = set(clm5_oaat_csv.loc[clm5_oaat_csv["location"]=="N"].name)
clm5_nml = clm5_oaat_csv.loc[clm5_oaat_csv["location"]=="N"]

In [252]:
for p in (clm5_nml_params & clm6_params):
    print("##", p)
    print("CLM5:", clm5_nml[clm5_nml["name"]==p]["CLM5 Default Value(s)"].item())
    print("CLM6:", clm6_pfile[p].values)

## fresh_snw_rds_max
CLM5: 204.526
CLM6: 400.0
## maximum_leaf_wetted_fraction
CLM5: 0.05
CLM6: 0.05
## upplim_destruct_metamorph
CLM5: 175
CLM6: 250
## decomp_depth_efolding
CLM5: 10
CLM6: 10.0
## jmaxb1
CLM5: 0.17
CLM6: [ nan 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17
 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17
 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17
 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17
 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17
 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17]
## interception_fraction
CLM5: 1
CLM6: 1.0
